In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
driver_path = r'C:\chromedriver-win64\chromedriver-win64\chromedriver.exe'
url = 'https://ca.trustpilot.com/review/www.amazon.com'


#### Online stores review scraping

In [9]:
# Driver path
driver_path = r'C:\chromedriver-win64\chromedriver-win64\chromedriver.exe'

# Set Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Optional: run in headless mode

# Initialize the Selenium WebDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# List of URLs to scrape
urls = {
    'amazon': 'https://ca.trustpilot.com/review/www.amazon.com',
    'ebay': 'https://ca.trustpilot.com/review/ebay.com',
    'alibaba': 'https://ca.trustpilot.com/review/www.alibaba.com',
    'walmart': 'https://ca.trustpilot.com/review/www.walmart.com',
    'bestbuy': 'https://ca.trustpilot.com/review/www.bestbuy.com'
}

# Function to scrape reviews from a given URL
def scrape_reviews(url, max_scrolls):
    wait = WebDriverWait(driver, 10)
    review_data = []

    driver.get(url)

    for scroll_count in range(max_scrolls):
        try:
            # Re-fetch the reviews container to avoid stale element
            reviews_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "styles_reviewsContainer__3_GQw")))

            # Scroll to the reviews container
            driver.execute_script("arguments[0].scrollIntoView();", reviews_container)
            time.sleep(2)  # Adjust sleep if necessary to allow reviews to load

            # Get the page source and parse with BeautifulSoup after scrolling
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Find all review containers on the page (updated class selector)
            reviews = soup.find_all('div', {'class': 'styles_reviewCard__9HxJJ'})

            # Print the number of reviews found in this scroll
            print(f"Number of reviews found: {len(reviews)}")

            # Extract the review text, rating, and consumer details for each review
            for review in reviews:
                try:
                    consumer_info = review.find('aside', {'class': 'styles_consumerInfoWrapper__KP3Ra'})

                    if consumer_info:
                        consumer_details = consumer_info.find('div', {'class': 'styles_consumerDetailsWrapper__p2wdr'})

                        consumer_name = consumer_details.find('a').find('span', {'class': 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}).text if consumer_details else 'Not Available'

                        # Extract consumer review number (handle missing elements)
                        consumer_review_no_element = consumer_details.find('div', {'class':'styles_consumerExtraDetails__fxS4S'}).find('span', 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l')
                        consumer_review_no = consumer_review_no_element.text if consumer_review_no_element else 'Not Available'

                        # Extract consumer country (handle missing elements)
                        consumer_country_element = consumer_details.find('div', {'class':'styles_consumerExtraDetails__fxS4S'}).find('div', 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua')
                        consumer_country = consumer_country_element.find('span').text if consumer_country_element else 'Not Available'
                    else:
                        consumer_name = 'Not Available'
                        consumer_review_no = 'Not Available'
                        consumer_country = 'Not Available'

                    consumer_review_info = review.find('section', {'class':'styles_reviewContentwrapper__zH_9M'})

                    if consumer_review_info:
                        consumer_rating = consumer_review_info.find('div', {'class':'styles_reviewHeader__iU9Px'}).find('div', {'class': 'star-rating_starRating__4rrcf star-rating_medium__iN6Ty'}).find('img')
                        rating_text = consumer_rating.get('alt') if consumer_rating else 'Not Available'

                        review_uploaded = consumer_review_info.find('div', {'class':'styles_reviewHeader__iU9Px'}).find('div', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH'}).find('time')
                        review_uploaded_info = review_uploaded.get('title') if review_uploaded else 'Not Available'

                        review_heading = review.find('div', {'class':'styles_reviewContent__0Q2Tg'}).find('a', {'class':'link_internal__7XN06 typography_appearance-default__AAY17 typography_color-inherit__TlgPO link_link__IZzHN link_notUnderlined__szqki'}).find('h2', {'class':'typography_heading-s__f7029 typography_appearance-default__AAY17'}).text if review.find('h2') else 'Not Available'

                        review_para = review.find('div', {'class':'styles_reviewContent__0Q2Tg'}).find('p', {'class':'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'}).text if review.find('p') else 'Not Available'
                    else:
                        rating_text = 'Not Available'
                        review_uploaded_info = 'Not Available'
                        review_heading = 'Not Available'
                        review_para = 'Not Available'

                    # Add review details to the list
                    review_data.append({
                        'consumer_name': consumer_name,
                        'consumer_review': consumer_review_no,
                        'consumer_country': consumer_country,
                        'date': review_uploaded_info,
                        'rating' : rating_text,
                        'review_topic': review_heading,
                        'review_content': review_para
                    })

                except Exception as e:
                    print(f"Error extracting review data: {e}")

            # If max scrolls have been reached or no more reviews found, stop scrolling
            if scroll_count == max_scrolls - 1 or len(reviews) == 0:
                break

            # Try to find the 'Next' link and scroll into view before clicking it
            try:
                next_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.button_button__T34Lr.pagination-link_next__SDNU4")))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_link)
                time.sleep(2)  # Allow the page to settle after the scroll
                driver.execute_script("arguments[0].click();", next_link)  # Click using JavaScript to avoid element interception
                print("Next page link clicked.")
                time.sleep(3)  # Wait for the next page of reviews to load

            except Exception as e:
                print(f"No more pages or next link not found: {e}")
                break

        except Exception as e:
            print(f"Error while scrolling: {e}")
            break

    return review_data

# Iterate over each URL and scrape reviews
all_reviews = []
for site_name, url in urls.items():
    print(f"Scraping reviews from {site_name}...")
    site_reviews = scrape_reviews(url, max_scrolls=15)

    # Add a 'site' column to identify which site the review came from
    for review in site_reviews:
        review['site'] = site_name

    all_reviews.extend(site_reviews)

# Close the driver after all scraping is done
driver.quit()

# Convert the list of reviews into a DataFrame
df = pd.DataFrame(all_reviews)

# Show the DataFrame
print(df)

# Save the reviews to a CSV file
df.to_csv('all_site_reviews_trustpilot.csv', index=False)

print("Scraping complete. Data saved to 'all_site_reviews_trustpilot.csv'.")


Scraping reviews from amazon...
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Error extracting review data: 'NoneType' object has no attribute 'text'
Error extracting review data: 'NoneType' object has no attribute 'text'
Error extracting review data: 'NoneType' object has no attribute 'text'
Error extracting review data: 'NoneType' object has no attribute 'text'
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Number of reviews found: 21
Next page link clicked.
Numb

In [10]:
df = pd.read_csv('all_site_reviews_trustpilot.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1531 entries, 0 to 1530
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   consumer_name     1530 non-null   object
 1   consumer_review   1531 non-null   object
 2   consumer_country  1531 non-null   object
 3   date              1531 non-null   object
 4   rating            1531 non-null   object
 5   review_topic      1531 non-null   object
 6   review_content    1531 non-null   object
 7   site              1531 non-null   object
dtypes: object(8)
memory usage: 95.8+ KB


       consumer_name consumer_review consumer_country  \
0  Vladimiro Mascaro       3 reviews               GB   
1           Lucy Loo       9 reviews               GB   
2                 Da      21 reviews               GB   
3        Vicki Study        1 review               GB   
4      Jeffrey Bruce       5 reviews               GB   

                                          date                  rating  \
0    Friday, October 11, 2024 at 06:25:37 p.m.  Rated 1 out of 5 stars   
1    Friday, October 11, 2024 at 09:57:32 a.m.  Rated 1 out of 5 stars   
2  Thursday, October 10, 2024 at 11:36:09 a.m.  Rated 5 out of 5 stars   
3    Friday, October 11, 2024 at 03:36:33 p.m.  Rated 1 out of 5 stars   
4  Thursday, October 10, 2024 at 05:09:02 p.m.  Rated 1 out of 5 stars   

                               review_topic  \
0             4 MONTHS OF TOTAL INCOPETENCE   
1                          Driver stealing    
2      Where is the negativity coming from?   
3    Absolutely appallin

In [12]:
df2 = pd.read_csv('all_reviews_with_full_nlp_analysis.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1531 entries, 0 to 1530
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   consumer_name        1530 non-null   object 
 1   consumer_review      1531 non-null   object 
 2   consumer_country     1531 non-null   object 
 3   date                 1531 non-null   object 
 4   rating               1531 non-null   object 
 5   review_topic         1531 non-null   object 
 6   review_content       1531 non-null   object 
 7   site                 1531 non-null   object 
 8   sentiment_score      1531 non-null   float64
 9   sentiment_magnitude  1531 non-null   float64
 10  entities             1451 non-null   object 
 11  entity_sentiments    1531 non-null   object 
 12  review_class         1531 non-null   object 
 13  positive_reasons     841 non-null    object 
 14  negative_reasons     1261 non-null   object 
 15  syntax_tokens        1531 non-null   o